### Note
This file is compilation all possible POS tags

In [4]:
# # Rename the files to gold and silver
# import argparse
# import os
# path = './data/train_20k/'
# count = 0
# for file in os.listdir(path):
#         count= count + 1
#         dst ="Silver_" + str(count) + ".csv"
#         src =file
#         os.rename(path+src,path+dst) 

In [54]:
ma_set = set()
word = None
with open('../data/multilingual_word_embeddings/pos_embedding_1_hot.475', 'r', encoding='utf-8') as f:
    # skip first line
    for i, line in enumerate(f):
        if i == 0:
            continue
        word, vec = line.split('@', 1)
        ma_set.add(word)
# ma_set

In [17]:
import re
def clean_ma(ma):
    ma = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []').replace(' ac','').replace(' ps','').replace('sgpl','sg').replace('sgdu','sg')
    ma = ma.replace('i.','inst.').replace('.','').replace(' ','')
    return ma
def IND(ma):
    indeclinable = ['ind','prep','interj','prep','conj','part','abs','ca abs']
    for i in indeclinable:
        if i in ma:
            ma = 'indecl'
    return ma
s = set()
for ma in ma_set:
    s.add(clean_ma(ma))
len(s)

244

In [47]:
df = pd.read_csv('../../../Documents/DCST/data/4.csv',sep=',')
# df.iloc[:,1]
# temp =[]
# for w in df.iloc[:,1]:
#     temp.append(transliterate(w, sanscript.WX, sanscript.SLP1))
# temp
list(df.iloc[:,1])

['vfkzARAm', 'latAnAm', 'ca', 'SAKAsu', 'vihagAH', 'kUjanti']

In [51]:
def allow(words,temp):
    ct = 0 
    for w,t in zip(words,temp):
        if w==t:
            ct+=1
    if ct == len(words) -1:
        return True
    else:
        return False

In [52]:
import os
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
data = pd.read_csv('../data/sanskrit_treebank/train/Gold_633.csv',sep=',')
def get_gold_POS_Tag(words):
    kautilya_id ='NA'
    path = '../../../Documents/DCST/data/4k_data_my_prepocessing/'
    files = os.listdir(path)
    gold_pos= ['NA']*len(words)
    for file in files:
        df = pd.read_csv(path+file,sep='\t',header = None)
        temp =[]
        if len(words) != len(df):
            continue
        for w in df.iloc[:,1]:
            temp.append(transliterate(w, sanscript.WX, sanscript.SLP1))
        words = [x.lower() for x in words]
        temp = [x.lower() for x in temp]
        if words == temp:
            kautilya_id = file
            for u,g in enumerate(df.iloc[:,2]):
                if g == g:
                    gold_pos[u] = g
            break
        elif allow(words,temp):
            kautilya_id = file
            for u,g in enumerate(df.iloc[:,2]):
                if g == g:
                    gold_pos[u] = g
        else:
            continue
            
    return gold_pos,kautilya_id
get_gold_POS_Tag(list(df.iloc[:,1]))
        
    

(['NP', 'NP', 'APY', 'NA', 'NP', 'NA'], '4.csv')

In [44]:
transliterate('nandigrAmam', sanscript.WX, sanscript.SLP1)

'nanqigrAmam'

In [58]:
import argparse
import os
path = '../data/sanskrit_treebank/'
languages_for_low_resource = ['san']
languages = sorted(list(set(languages_for_low_resource)))
#
splits = ['train','dev','test']
lng_to_files = dict((language, {}) for language in languages)
for language, d in lng_to_files.items():
    for split in splits:
        d[split] = []
    lng_to_files[language] = d
sub_folders = os.listdir(path)
for sub_folder in sub_folders:
    folder = os.path.join(path, sub_folder)
    files = sorted(os.listdir(folder))
    for file in files:
        full_path = os.path.join(folder, file)
        lng_to_files[language][sub_folder].append(full_path)
###################################
# # # # Add silver in train:
# counter = 0
# amount = 2000
# for f in os.listdir('../data/train_20k/'):
#     full_path = os.path.join('../data/train_20k/', f)
#     lng_to_files[language]['train'].append(full_path)
#     counter += 1
# #     if counter >= amount:
# #         break
    
word_set = set()
for language, split_dict in lng_to_files.items():
    posi =set()
    for split, files in split_dict.items():
        sentences = []
        num_sentences = 0
        for file in sorted(files):
#             if split =='train':
#                 print(file)
            
            flag = 0
            data = pd.read_csv(file,sep=',')
            gold_pos,kautilya_id = get_gold_POS_Tag(list(data.iloc[:,1]))
            if kautilya_id == 'NA':
                continue
            num_sentences += 1
            k=0
            with open(file, 'r') as file:
                for line in file:
                    new_line = []
                    line = line.strip()
#                     print(line)
                    if flag == 0:
                        flag = 1
                        continue
                    tokens = line.split(',')
                    id = str(int(tokens[0]) + 1)
                    word = tokens[1]
                    word_set.add(word)
                    ############################
                    # To add rule features id of rule is given
                    my_data_id = file.name.replace('../data/sanskrit_treebank/','').replace('.csv','')\
                    .replace('../data/train_20k/','').replace('train/','').replace('dev/','').replace('test/','')
                    lemma = tokens[2]
                    pos = tokens[3]
#                     tag.add(pos)
                    if tokens[6]:
                        arc_tag = tokens[5]
                        head = str(int(tokens[6]) + 1)
                    else:
                        arc_tag = 'root'
                        head = '0'
                    
                    new_line = [id, word ,pos,get_modified_coarse(pos),gold_pos[k],my_data_id,head, arc_tag,kautilya_id]
                    k+=1
                    posi.add(pos)
#                     print(new_line)
                    sentences.append(new_line)
            sentences.append([])
            print(num_sentences)
#         ud_pos_ner_dp_
        print('Language: %s Split: %s Num. Sentences: %s ' % (language, split, num_sentences))
        if not os.path.exists('data'):
            os.makedirs('data')
#         ud_pos_ner_dp_
        write_data_path = '../../../DCST_scratch/data/data_with_gold_pos_' + split + '_' + language
        print('creating %s' % write_data_path)
        with open(write_data_path, 'w') as f:
            for line in sentences:
                f.write('\t'.join(line) + '\n')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613


In [19]:
def get_case_from_244(ma):
    indeclinable = ['ind','prep','interj','prep','conj','part','indecl']
    case_list = ['nom','voc','acc','inst','dat','abl','loc','i','g']
    gender_list = ['n','f','m','*']
    person_list = ['1','2','3']
    no_list = ['du','sg','pl']
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    # Remove active passive
    case=''
    if ma == 'comp':
        case ='comp'
    for tag in indeclinable:
        if tag in ma:
            case= "IND"

    # Get case
    if case =='':
        for tag in case_list:
            if tag in ma:
                if tag == 'i':
                    if tag+'sg' in ma or tag+'du' in ma or tag+'pl' in ma:
                        case = 'i'
                elif tag == 'g':
                    if tag+'sg' in ma or tag+'du' in ma or tag+'pl' in ma:
                        case = 'g'
                else:
                    case = tag
    if case == '':
        if 'adv' in ma:
            case = 'adv'
    
    if case =='':
        for tag in no_list:
            if tag in ma:
                case = 'FV'
    if case=='':
        for tag in no_list:
            if tag in ma:
                case = 'FV'
    if case=='':
         for tag in person_list:
            if tag in ma:
                case = 'FV'
    if case=='':
        case='IV'
        
    return case
s = []
for m in ma_set:
    s.append([m,get_case_from_244(m)])

In [61]:
import json
indeclinable = ['ind','prep','interj','prep','conj','part','abs','ca abs']
case_list = ['nom','voc','acc','i','inst','dat','abl','g','loc']
gender_list = ['n','f','m','*']
person_list = ['1','2','3']
no_list = ['du','sg','pl']
pops = [' ac',' ps']
def get_modified_coarse(ma):
    ma = ma.replace('sgpl','sg').replace('sgdu','sg')
    with open('../../L2S/files/coarse_to_ma_dict.json', 'r') as fh:
        coarse_dict = json.load(fh)
    for key in coarse_dict.keys():
        if ma in coarse_dict[key]:
            return key
s=[]
for m in ma_set:
    s.append([m,get_modified_coarse(m)])
sorted(s)

[['abl. du. f.', 'Noun'],
 ['abl. du. m.', 'Noun'],
 ['abl. du. n.', 'Noun'],
 ['abl. pl. f.', 'Noun'],
 ['abl. pl. m.', 'Noun'],
 ['abl. pl. n.', 'Noun'],
 ['abl. sg. *', 'Noun'],
 ['abl. sg. f.', 'Noun'],
 ['abl. sg. m', 'Noun'],
 ['abl. sg. m.', 'Noun'],
 ['abl. sg. n', 'Noun'],
 ['abl. sg. n.', 'Noun'],
 ['abl.sg.m', 'Noun'],
 ['abs.', 'IV'],
 ['acc. du. *', 'Noun'],
 ['acc. du. f.', 'Noun'],
 ['acc. du. m.', 'Noun'],
 ['acc. du. n.', 'Noun'],
 ['acc. pl. *', 'Noun'],
 ['acc. pl. *.', 'Noun'],
 ['acc. pl. f.', 'Noun'],
 ['acc. pl. m.', 'Noun'],
 ['acc. pl. n.', 'Noun'],
 ['acc. sg. *', 'Noun'],
 ['acc. sg. *.', 'Noun'],
 ['acc. sg. f', 'Noun'],
 ['acc. sg. f.', 'Noun'],
 ['acc. sg. m.', 'Noun'],
 ['acc. sg. n', 'Noun'],
 ['acc. sg. n.', 'Noun'],
 ['adv.', 'adv'],
 ['aor. [1] ac. du. 3', 'FV'],
 ['aor. [1] ac. pl. 3', 'FV'],
 ['aor. [1] ac. sg. 1', 'FV'],
 ['aor. [1] ac. sg. 3', 'FV'],
 ['aor. [2] ac. sg. 3', 'FV'],
 ['aor. [4] ac. du. 2', 'FV'],
 ['aor. [4] ac. sg. 3', 'FV'],
 ['ao

In [60]:
print(get_modified_coarse('acc. pl. m'))

None


In [32]:
import re
def get_case(ma):
        indeclinable = ['ind','prep','interj','prep','conj','part']
        case_list = ['nom','voc','acc','i','inst','dat','abl','g','loc']
        gender_list = ['n','f','m','*']
        person_list = ['1','2','3']
        no_list = ['du','sg','pl']
        pops = [' ac',' ps']
        ma=ma.replace('sgpl','sg').replace('sgdu','sg')
        temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
        temp = temp.split('.')
        if temp[-1] == '':
            temp.pop(-1)
        # Remove active passive
        case=''
        no=''
        person=''
        gender=''
        tense=''
        coarse=''
        for a,b in enumerate(temp):
            if b in pops:
                temp.pop(a)
        # Get gender
        for a,b in enumerate(temp):
            if b.strip() in gender_list:
                gender = b.strip()
                temp.pop(a)
        # Get case
        for a,b in enumerate(temp):
            if b.strip() in case_list:
                case = b.strip()
                temp.pop(a)
        if case!= '':
            coarse ='Noun'
        # Get person
        for a,b in enumerate(temp):
            if b.strip() in person_list:
                person = b.strip()
                temp.pop(a)
        # Get no
        for a,b in enumerate(temp):
            if b.strip() in no_list:
                no = b.strip()
                temp.pop(a)
        # Get Tense
        for b in temp:
            tense=tense+ ' '+b.strip()
        tense=tense.strip()
        
#         print(tense)
        if tense == 'adv':
            coarse = 'adv'
        for ind in indeclinable:
            if tense == ind:
                coarse = 'Ind'
        if tense == 'abs' or tense == 'ca abs':
            coarse = 'IV'
        if tense!='' and coarse=='':
            if person !='' or no!='':
                coarse= 'FV'
            else:
                coarse = 'IV'
        if case == 'i':
            return 'inst'
        
        if case !='':
            return case
        else:
            return coarse

In [6]:
def variation_III(ma):
    ##############################################
    # Description:
    # (1) Active passive is removed
    # (2) Tense [*] is removed
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
    temp = temp.split('.')
    if temp[-1] == '':
        temp.pop(-1)
    # Remove active passive
    case=''
    no=''
    person=''
    gender=''
    tense=''

    for a,b in enumerate(temp):
        if b in pops:
            temp.pop(a)
    # Get gender
    for a,b in enumerate(temp):
        if b.strip() in gender_list:
            gender = b.strip()
            temp.pop(a)
    # Get case
    for a,b in enumerate(temp):
        if b.strip() in case_list:
            case = b.strip()
            temp.pop(a)
#     if case == 'i':
#         case = 'inst'
    # Get person
    for a,b in enumerate(temp):
        if b.strip() in person_list:
            person = b.strip()
            temp.pop(a)
    # Get no
    for a,b in enumerate(temp):
        if b.strip() in no_list:
            no = b.strip()
            temp.pop(a)
    # Get Tense
    for b in temp:
        tense=tense+ ' '+b.strip()
    tense=tense.strip()
    var=''

    if case:
        t = 'case=%s '%case
        var=var+t
    if gender:
        t = 'gender=%s '%gender
        var=var+t
    if tense:
        t = 'tense=%s '%tense
        var=var+t
    if no:
        t = 'no=%s '%no
        var=var+t
    if person:
        t = 'person=%s '%person
        var=var+t
    return var.strip()
variation_III('pp. . sg. ')


'tense=pp no=sg'

In [4]:
import re
def variation_IV(ma):
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
    temp = temp.split('.')
    if temp[-1] == '':
        temp.pop(-1)
    # Remove active passive
    case=''
    no=''
    person=''
    gender=''
    tense=''
    coarse=''
    for a,b in enumerate(temp):
        if b in pops:
            temp.pop(a)
    # Get gender
    for a,b in enumerate(temp):
        if b.strip() in gender_list:
            gender = b.strip()
            temp.pop(a)
    # Get case
    for a,b in enumerate(temp):
        if b.strip() in case_list:
            case = b.strip()
            temp.pop(a)
    if case!= '':
        coarse ='Noun'
#     if case == 'i':
#         case = 'inst'
    # Get person
    for a,b in enumerate(temp):
        if b.strip() in person_list:
            person = b.strip()
            temp.pop(a)
    # Get no
    for a,b in enumerate(temp):
        if b.strip() in no_list:
            no = b.strip()
            temp.pop(a)
    # Get Tense

    for b in temp:
        tense=tense+ ' '+b.strip()
    tense=tense.strip()
    if tense == 'adv':
        coarse = 'adv'
        
    for ind in indeclinable:
        if tense == ind:
            coarse = 'Ind'

    if tense!='' and coarse=='':
        if person !='' and no!='':
            coarse= 'FV'
        else:
            coarse = 'IV'
    var=''
    if case:
        t = 'case=%s '%case
        var=var+t
    if gender:
        t = 'gender=%s '%gender
        var=var+t
    if tense:
        t = 'tense=%s '%tense
        var=var+t
    if no:
        t = 'no=%s '%no
        var=var+t
    if person:
        t = 'person=%s '%person
        var=var+t
    coarse = get_modified_coarse(ma)
    if coarse:
        t = 'coarse=%s '%coarse
        var=var+t
    return var.strip();
l = []
# for ma in ma_set:
#     l.append(ma+' = '+variation_IV(ma))
variation_IV('pp. . sg. ')

'tense=pp no=sg coarse=FV'

In [25]:
# onehot_encoded[0]
count =0 
with open("pos_embedding_1_hot.475", "w") as f:   
    f.write(str(len(tag))+' '+str(475)+'\n')
    for word in tag:
        word=word+'@'
        f.write(word) 
        for v in onehot_encoded[count]:
            f.write(' '+str(v))
        f.write('\n')
        count = count + 1
print(count)

475
